# E91 QKD

In [ ]:
# Name: Brian Bahk
# Name: Minjae Cho
# Name: Elijah Olive

In [ ]:
# Install qiskit
!pip install qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 1.8 MB/s eta 0:00:00


In [ ]:
#@title #QC

from qiskit import QuantumCircuit
from qiskit.primitives.sampler import Sampler
import random
import numpy as np

class InputError(Exception):
    def __int__(self, expression, message):
        self.expression = expression
        self.message = message

class QC:

    def __init__(self):
        self.qc = QuantumCircuit(2, 2)

        # Create a Psi- state
        self.qc.h(1)
        self.qc.cx(1, 0)
        self.qc.x(1)
        self.qc.z(1)

        self.measured = 0

        self.alice_alpha = 0
        self.alice_beta = 0
        self.bob_alpha = 0
        self.bob_beta = 0

    def prepareHV(self):
        # Add a measurement to both qubits
        self.qc.measure([0, 1], [0, 1])

        # Execute using the Sampler primitive
        sampler = Sampler()
        job = sampler.run(self.qc, shots=1)
        result = job.result()
        ab = int(str(result.quasi_dists[0])[1])
        if ab == 1:
            self.alice_alpha = 1
            self.alice_beta  = 0
            self.bob_alpha = 0
            self.bob_beta  = 1
        elif ab == 2:
            self.alice_alpha = 0
            self.alice_beta  = 1
            self.bob_alpha = 1
            self.bob_beta  = 0

    def prepareDA(self):
        # Put hadamard before measuring to measure in D/A basis
        self.qc.h(0)
        self.qc.h(1)

        # Add a measurement to both qubits
        self.qc.measure([0, 1], [0, 1])

        # Execute using the Sampler primitive
        sampler = Sampler()
        job = sampler.run(self.qc, shots=1)
        result = job.result()
        ab = int(str(result.quasi_dists[0])[1])
        # Psi- = 1/2sqrt2 * (|AD> - |DA>)
        if ab == 1: # 01; DA
            self.alice_alpha = 1/np.sqrt(2)
            self.alice_beta  = 1/np.sqrt(2)
            self.bob_alpha = 1/np.sqrt(2)
            self.bob_beta  = -1/np.sqrt(2)
        elif ab == 2: # 10; AD
            self.alice_alpha = 1/np.sqrt(2)
            self.alice_beta  = -1/np.sqrt(2)
            self.bob_alpha = 1/np.sqrt(2)
            self.bob_beta  = 1/np.sqrt(2)

    # Call after preparing (aka measuring one of the qubits)
    def getAliceKeyHV(self, probDarkCount):
        if self.measured == 0:
            QC.prepareHV(self)
            self.measured = 1

        if self.alice_alpha == 1 and self.alice_beta == 0:
            return "H"
        elif self.alice_alpha == 0 and self.alice_beta == 1:
            return "V"
        else:
            return "N"

    def getAliceKeyDA(self, probDarkCount):
        if self.measured == 0:
            QC.prepareDA(self)
            self.measured = 1

        if self.alice_alpha == 1/np.sqrt(2) and self.alice_beta == 1/np.sqrt(2):
            return "D"
        elif self.alice_alpha == 1/np.sqrt(2) and self.alice_beta == -1/np.sqrt(2):
            return "A"
        else:
            return "N"

    def measureHV(self, probDarkCount):
        if self.bob_alpha == 1 and self.bob_beta == 0:
            return "H"
        elif self.bob_alpha == 0 and self.bob_beta == 1:
            return "V"
        else:
            return "N"

    def measureDA(self, probDarkCount):
        if self.bob_alpha == 1/np.sqrt(2) and self.bob_beta == 1/np.sqrt(2):
            return "D"
        elif self.bob_alpha == 1/np.sqrt(2) and self.bob_beta == -1/np.sqrt(2):
            return "A"
        else:
            return "N"

    def eveSendH(self):
        self.bob_alpha = 1
        self.bob_beta  = 0

    def eveSendV(self):
        self.bob_alpha = 0
        self.bob_beta  = 1

    def eveSendD(self):
        self.bob_alpha = 1/np.sqrt(2)
        self.bob_beta  = 1/np.sqrt(2)

    def eveSendA(self):
        self.bob_alpha = 1/np.sqrt(2)
        self.bob_beta  = -1/np.sqrt(2)
    def bell_measurement(self, basis_a, basis_b):
        # Reset the circuit to start fresh for each measurement
        self.qc.reset([0, 1])

        # Prepare the Psi- state
        self.qc.h(1)
        self.qc.cx(1, 0)
        self.qc.x(1)
        self.qc.z(1)

        # Apply basis change depending on input
        if basis_a == 'D' or basis_a == 'A':
            self.qc.h(0)  # Hadamard on Alice's qubit for D/A basis
        if basis_b == 'D' or basis_b == 'A':
            self.qc.h(1)  # Hadamard on Bob's qubit for D/A basis

        # Measure
        self.qc.measure([0, 1], [0, 1])

        # Execute the circuit
        sampler = Sampler()
        job = sampler.run(self.qc, shots=1)
        result = job.result()
        ab = int(str(result.quasi_dists[0])[1])  # Extract the result
        return ab
    def calculate_correlations(self, shots=1000):
      settings = [('H', 'V'), ('H', 'D'), ('D', 'V'), ('D', 'D')]
      results = {setting: 0 for setting in settings}
      for _ in range(shots):
        for setting in settings:
            results[setting] += self.bell_measurement(*setting)
      return results

In [ ]:
n = 1000 # number of photons

In [ ]:
qcArray = [QC() for i in range(n)]

In [ ]:
# Eve selects a subsample of photons from Alice to measure.
# interceptIndex should be a string of n characters.
# Use the convention '0'=ignored, '1'=intercepted
interceptIndex = ""
# TODO: Put your code here.
for i in range(n):
  if random.randint(0,1) == 0:
    interceptIndex += "0"
  else:
    interceptIndex += "1"
print(interceptIndex)

0011100011100110111110100110000100110100110101010000000000001001101011101111110001011000000100011110000010011000000111110110111000001110001011100000010100010001111100010100000110011000111100101110010011110101101110000101110101101011100011101111010001000001011011100110100110111001101111000111011000011111101100001100100101111110100110011111010111110100111111111111110101110110000001010011000100000110100110011000011011010110000011000001111001010101010111010100010110100010111011000000001010001110010100001100000010000111111101011000001000011000100101010110101111101110101001010100011111100110000100001110110111101010001110100110111110101000000010111111100000011100011101001001001101100101000110001101111101111111000100001110011001111010000101110001100011101011000001011111010011010111111011010111111000011001101000011100011010011110100001011011010000011011110101001101010000011001011010011100000001111110110001101110010001011101101110100110010100001111101100011000000100010111001000111010110010110000

In [ ]:
# Eve chooses a basis to measure each intercepted photon.
# basisEve should be a string of n characters.
# Use the convention '+'=H/V, 'x'=D/A, ' '=not measured
basisEve = ""
# TODO: Put your code here.
for i in range(n):
  if interceptIndex[i] == '1':
    if random.randint(0,1) == 0:
      basisEve += "+"
    else:
      basisEve += "x"
  else:
    basisEve += ' '
print(basisEve)

  xxx   xxx  x+ +++xx +  +x    +  +x x  xx + x x            +  xx x ++x xx++xx   + +x      x   x+x+     +  +x      +x+x+ +x +++     +x+   x x+x      + x   +   xxx++   + x     +x  x+   x++x  x +++  x  xx+x + xx x+x    + xxx + x+ + xxx   x+x ++++ x   +     x ++ xx+  ++ x  ++ +x+  xx ++x+   +++ xx    x+++xx +x    ++  +  + +++xxx x  x+  x+x+x x xx+++ +  x+xxxx++x+xx++ + ++x xx      x +  ++   +     xx +  ++  xx    ++ ++ x xx     +x     xx+x  x + x + x xx+ + +   x x+ x   x xx+ xx        x +   xxx  x x    x+      x    +xx+xxx x x+     +    x+   x  + + x x+ + ++x+x +xx + +  x + +   xxxxxx  x+    x    ++x ++ x+xx x +   ++x +  xx +++x+ x +       + xxx+xx+      xxx   +x+ +  +  x  x+ +x  + +   +x   x+ +x+xx ++++x++   x    xxx  +x  xxxx +    + x++   xx   +x+ + xx     x +x+++ x  +x + +x+xxx x+ x xx+xx+    xx  xx +    xx+   xx +  ++xx x    x ++ x+ +     ++ xx+x + +  +x x x     xx  + ++ +  +xx       ++++xx +x   +x +++  +   x ++x x+ ++x +  xx  x +    +xxxx ++   ++      +   x x+x  +   +x+ + x+  + xx    

In [ ]:
# Eve performs a measurement on each photon.
# outcomeEve should be a string of n characters.
# Use the convention 'H','V','D','A', ' '=not measured
outcomeEve = ""
# TODO: Put your code here.
for i in range(n):
    if basisEve[i] == '+':
        outcomeEve += QC.getAliceKeyHV(qcArray[i], .5)
    elif basisEve[i] == 'x':
        outcomeEve += QC.getAliceKeyDA(qcArray[i], .5)
    else:
      outcomeEve += ' '
print(outcomeEve)

<ipython-input-3-bf18f3a66c6f>:60: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()
<ipython-input-3-bf18f3a66c6f>:36: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()


  DAD   AAA  AV VHHAD H  HA    V  HD A  DA V A A            V  DD A VVD DDVHDD   V VA      D   AHAV     V  HD      HDHAH VD VHH     HAV   A DVD      H A   H   DAAVH   H A     HA  AH   AHHD  A VVH  D  ADHA V DA DHD    H AAD V DV H AAD   AVD VHVV A   H     D VH AAH  VH D  VH HDV  AD HVAV   HHH DD    DHVHDA VD    VH  V  V VHVDAA D  DH  AHDHA A AAVHH V  DVDDADVHAHDAVV H HHA AD      D V  HV   V     DD H  VV  AA    VV VH D AD     VA     AAVD  D H A H A ADH V V   A AV A   D DDH DA        A V   AAA  D A    DH      D    VAAHADA A AH     V    DV   A  V H D DV H HVAVD HAD H H  A H V   ADAAAD  DV    A    VVA HH DVDD A V   HHD H  AA VVVAH A H       V DAAHDAV      ADD   HAH H  H  A  DH HA  V H   VA   DV HAHDD VVVHAHH   A    DAD  HD  ADAD V    H DHV   DA   HDH H AA     A HAHVH A  VA V HDVDDA DV A DAVADH    DA  AD V    ADH   DA V  HVDA A    D VH AV H     HH DAHA V V  VD D A     DA  H VV H  HDA       VVVVAD HD   HD HVH  H   D VVA DV VHA H  AA  D H    HAAAA VV   VV      V   D AHD  H   HDV V DV  H AA    

In [ ]:
# # Eve resends photons to Bob.
# # Be sure to handle the cases in which Eve gets an invalid measurement.
# # TODO: Put your code here.
# for i in range(n):
#   if outcomeEve[i] == "H":
#     QC.eveSendH(qcArray[i])
#   elif outcomeEve[i] == "V":
#     QC.eveSendV(qcArray[i])
#   elif outcomeEve[i] == "D":
#     QC.eveSendD(qcArray[i])
#   elif outcomeEve[i] == "A":
#     QC.eveSendA(qcArray[i])
#   elif outcomeEve[i] == "N" or outcomeEve[i] == 'M':
#     chooseRand = random.randint(0, 3)
#     if chooseRand == 0:
#       QC.eveSendH(qcArray[i])
#     elif chooseRand == 1:
#       QC.eveSendV(qcArray[i])
#     elif chooseRand == 2:
#       QC.eveSendD(qcArray[i])
#     elif chooseRand == 3:
#       QC.eveSendA(qcArray[i])
#     #send random if N also choose based off of outcome if blank no change

In [ ]:
# Alice chooses the encoding basis for each key bit.
# This should be a string of '+'s and 'x's with '+'=H/V, 'x'=D/A.
basisAlice = ""
for index in range(n):
  if random.randint(0,1) == 0:
    basisAlice += "+"
  else:
    basisAlice += "x"
print("basisAlice  = " + basisAlice)

basisAlice  = xxx++xxx+++x++x+x++xx+xxxx+x+xx+++xxx+x+x+++xxxxx+x+xxx+xx+x+xx+xx+++x++xx+xx+x+xx++xxxx+++++xxxx++xxx++xxxxxx+x+x+x+++xxxx+xx++x+++++xxx+xx+xx++xx+x+++xxx++++x++x++xx+x++x+x++xx+++++xxxxx+xxx+x+xx+xx+++++x+x+xx+x+x+xxx+xx+xxx++++xx++x+x++xxxx++x+++++xx+++++xxxx+xxxx+xxx+++++x+x+xxxx+xxx++xx++x+x+x+x+x++x++xxxxx+xxx++x++xx+++xxx+x+x++x+xx+x++xxx++xxx++x+x++xxxx++++xxx+x++++xx+xx+xx++x++x++++x+x+++x++xx++x+xx+x+x++++xxx+xx++xx+++xxx+++xx+xx+xxx+++xx+x+x+xxxx+xx+++++x++xxxx+++++++++++xx++xx+xx++++xxxxxx+++++xxx+++++xxxx+x+x+++xx+++x+xxx+xxx++++x+xxxx+++++x+x+x+++xx+++xxxxx++++++xx++++++x+x+x++xxxxxx++xx++xxxx++xx+xxx++x+x++x+x++xxxxx+++x++x+xx++x+++xx+xx+xx++xx++xxx+x+++++xxx+++x+xxxxx+xxx+xxxxx++xxxxxx+xx++xxxx+x++x+x+++x++xxx+xxx++x+xx++x+xxx+xx++xx+x+xx+xx+xxx+++xxxx++xxx+++x++xx++xxxxxx++x+++++++xxxx+++xx+x+++++x++xx++xxxx++++x+++++x+x++x+xx+xx+++xxx++++x++x++xxx++x+xxx+xx++x+xxxxx++x++x++xxx+++xxxx+x+xx+xx+++xxxxx+x+x+++++++xx++xxx+xxx++x++xxxx+++x+xxx++xxx+x+x+x+x+xx+

In [ ]:
# Alice measures Bell State based on chosen basis
# Alice gets the raw key.
polarizationAlice = ""
for i in range(n):
  if basisAlice[i] == "+":
    polarizationAlice += QC.getAliceKeyHV(qcArray[i], .5)
  elif basisAlice[i] == "x":
    polarizationAlice += QC.getAliceKeyDA(qcArray[i], .5)
print("keyAlice  = " + polarizationAlice)

<ipython-input-3-bf18f3a66c6f>:60: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()
<ipython-input-3-bf18f3a66c6f>:36: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()


keyAlice  = DDDNNDADNNNDVNNHNHHADVNDDNNAVDAVHHNDANAVDNHVAADADHDHDDDHDDHDVDANDANVVNNVDDVNDNAHDNHVAAADHHHNVADANNVADAHHNDANDDHDVAVNNHNNANDVNNHVDVVVHNNAAHADNNDVVDDVAHHNAADHVVHDNNNHVAAHANHDVDVHADHNHVHAANNDVAADVNHDANAANNHNHNHDNADHDHDVDNANADHNDDVVHVAANVDHAVNANNNVVAHHHHVDDHVNVVNDAAHAANNHDAAVHVHNNVANDANNNNAAVHNNVNDVAVANNVNNNAVNADDDNHAANVHNHVNNNNNDDDVDHAHNNNNAVAVNANNHHNAANVDNANVNANDNVVHNDNHAVNNVADHAANANVHNVHDHVHHDHDNNVNVHNNVVANDDVDVNHVHVDAANDAHVANNHHADDNNVDDHDDHDAAHHNDANNHNHNAAANDAVVNVVANVDDNANNVHHVHVVVNDNVVAANADHNVNDDAADNVHVHVDDAVHHVNANADNDNDNHHDAVVVAVAADVADANVVVDHADADVVHVHNNNNAHNNDNVHHDADNAVHHHNNAANNVVNVDVDVAHVDANNADHHDDVNDAADVHADHNDDHHANAVVNVAHHADNDAHHVAVVDNAAHNAVHVDAHDANDDVVNAHVNDDHAVNHHNNDNNVHNVNDAANNADDNNDNANNNDNNNNAHNDHHADDDVDNNDHNNHHANNDDNHDDDHVDHNDHVDNAAAHDNVHAANDHDDVADHANNHVNADNAHVANDVNNAHNNDNVDANADNVVAVNNVHNNANAAHHNDNHAVNNHVDVHNDNHADDAVNVVNVNVHHDVDVHNHADNNAVVHNDAVNVNDNHDHHADAVHNVNNAHDAHNAVAAAADVVNVVANVNDAHVHDDNNHAHNDHDDVVVADDNDVNNDHVVNNHVDAHHDDAHAAANHNVHADNNHVHAVDNAVHDAAHDVDHAHAHDNHVH

In [ ]:

# for i in range(n):
#   if basisAlice[i] == "+":
#     polarizationAlice += QC.getAliceKeyHV(qcArray[i], .5)
#   elif basisAlice[i] == "x":
#     polarizationAlice += QC.getAliceKeyDA(qcArray[i], .5)


In [ ]:
# Alice infers the raw key.
# keyAlice should be a string of n characters.
# Use the convention '0', '1', '-'=invalid measurement
keyAlice = ""
# TODO: Put your code here.
for i in range(n):
    if basisAlice[i] == '+':
        if polarizationAlice[i] == 'H':
            keyAlice += '0'
        elif polarizationAlice[i] == 'V':
            keyAlice += '1'
        else:
            keyAlice += '-'
    elif basisAlice[i] == 'x':
        if polarizationAlice[i] == 'D':
            keyAlice += '0'
        elif polarizationAlice[i] == 'A':
            keyAlice += '1'
        else:
            keyAlice += '-'
print("keyAlice      = " + keyAlice)

keyAlice      = 000--010---01--0-00101-00--1101100-01-110-011101000000000000101-01-11--1001-0-100-011110000-1101--110100-01-0000111--0--1-01--0101110--11010--011001100-11001100---011101-001010100-01011--011101-001-11--0-0-00-10000010-1-100-00110111-10011-1---111000010001-11-011011--00111010--11-01----1110--1-01111--1---11-1000-011-10-01-----00010010----1111-1--00-11-10-1-1-1-0-110-0-011--110011-1-10-1000100000--1-10--111-00101-0101011-01011--00100--100000001100-01--0-0-111-0111-111-100-1--10010111-0-1111-100-1-00110-101010011001-1-10-0-0-0001111111101101-11100101011010----10--0-100010-11000--11--11-1010110101--1000001-011010100-00001-111-110010-010011110-110-110101001-0011-101-00011-00--0--10-1-011--100--0-1---0----10-000100010--00--001--00-00000100-0010-11100-1011-000011001--01-10-1011-01--10--0-101-10-1111--10--1-1100-0-011--01010-0-010011-11-1-10001010-010--1110-011-1-0-000010110-1--10010-111111011-111-1-0101000--010-0000111100-01--0011--0101000010111-0-1010--010110-1100110010010100

In [ ]:
# Bob   --------------------------------------------

# Bob chooses a basis to measure each photon.
# This should be a string of '+'s and 'x's with '+'=H/V, 'x'=D/A.
basisBob = ""
for index in range(n):
  if random.randint(0,1) == 0:
    basisBob += "+"
  else:
    basisBob += "x"

print("basisBob    = " + basisBob)

basisBob    = ++xxxxxx+x+++++xx+x++x+xxxxxx+xxx+xx++xxx+xx+xxx++xxx+x+xx++x++x+x+x++x+x+xx++++xxxx++xx+xx+++x++x+xxxxxx+x+++x++++xx+xx++xxxx+++x+x+x++xx+x+x++xxxxx+x++x+++x+++x++x+xx++++x++x+x+xxx++x+x+x++x+x++xx+xx+++x++x++++xx++xxx++xx+xxx+++++xxx++++++x+x+++++xxx++xx+xxx+++xx++x++x+x++xx++xxx+xx++xx+x++x++x++xxx+xx+x++++++x+xxx+xxxxx++x++x+xxx+xxx+++xxx+x+xx+xxx++x++xx++xxx++xxx++xxx+xx+xxx+xx+++xx++x+x+++x+xx++xxx+x+x+x++x++xxxx+x+x++xxx++xxxx++x+x+++xxx+x++++xx+xx+xxxx++xx+x++xx++xxxxxx+xxx+xx++x+x++xxxx+x+xxxx+xxxx+++xxx++xx++xx+xxxx+xxx++xxx++++xxxx+x+++x+xx+xx++x+x++x+x+++x+x++xxx+xxxxx++xx++x+++xx+xx+++xxxxxxx++x+xx+++xx+x+x+x++x+++xxxx+x++x++x+++xx++xx+x++++x++x++x+x+xxx+xxxxxx++xx+x++x+xxxxxx+++++xxxxx+++xxx+xxx+xxx+++xxx+++xx+x++xx++++xxx+xxxxx+++x+xx+xxx+xx+xx++x+++xx++xx+++x++x+x+xxxxx++x++++xx+x+x+++xx++x++x+++++++x++xxxx+xxx+++xx+xxx+x+++x++xxx+++xx++xxx+xx+++++x+++xx++++x++++++++xx+xx++xxx++x++++++xx+++++xx+x++x++++x+x+++xx+++xxxxxxx++xxxx++++x+x++xxxxx+++x+x+x+x++x+xx

In [ ]:
# Bob performs a measurement on each photon.
# Use the methods of the Photon class to measure each photon.
# outcomeBob should be a string of n characters.
# Use the convention 'H','V','D','A', ' '=not measured
outcomeBob = ""
for i in range(n):
    if basisBob[i] == '+':
        outcomeBob += QC.measureHV(qcArray[i], .5)
    elif basisBob[i] == 'x':
        outcomeBob += QC.measureDA(qcArray[i], .5)
print("outcomeBob  = " + outcomeBob)

outcomeBob  = NNADAADANDNNHNHNNVNNNNVAANDDNNDNNVNANNDNANNNNDADNVANANAVAAVNNNNANDNNHHAHANNNNNNVANNNNNDAVNNNHNANVDHDADNNNNDVNNNNHNHNAVDNNHANNNVHNNHNVDHNDNNANNNHNAANDVNNNDNVHNVNNDHVNNDNNNVNNNHNNAVDNNVNDVNNNNNAHNVNDANDDNVNNHVANNNVANNHANDNNANHAANHVHNNANAVNHNNHNHNHNVVVNNANVNAHNNANNVDDHVNNNDHNHVANHNDADVNDHNDNVNVHANHDHNANNVADNNNNNNNHNNDNNVNNNNNNNDNNAHANDVDNAVNHDNDNNVNNHDDAHNANNNNNVADNHVNANVNNDAHDAVDDANNNVVHNAVHNVAVNNAHNNVHNNNNDNAHAHHNHVNADDNANNHNNDNVNAADDHNAVANVNDDNVDNNNVNNVNDNDDADHHDNHDNHAAVNADNNNNVNNNNANHHDNDNNNANDNANDANNVNNNANNHNNNNNNDNNADNDNNANNNNNHDDAHNNNDNNNNNNNNAHNNHNNNHANNNNAVNVVNDNNNHNNNNADDDAHHANNHAHNVNANNNNNVNAANAADNNNVDAVVNANVDNDHNHHDVVNANADVNHNNHNANNVADHVNANNNNNNAHHNNVNVANNDNNNNANANNHNNHNNNDHDDAAANNVNVNAANNNVNVNANVDAANNADNNVNANVNNDANNVNAAVHNVNANHADDDDVNVNVDDNANANNDNNDVHNHNNANNVNDVNHANNNNNNDNADNNNNHHNHADHNNANHNDNVNAVVDHNNVHNHNHNDNDANDNAHHVNDHNNAHAHVVNNNDNDHHVNANHANANDNNVVNNDHVVNNHNVNDVNNHNNNNANHNNHNANNNNNHVNNVHNDVVNVNANHNNNAHNHVDNNHHNDNHNNNNAADNDNNDNNNVNNHNVNVNNANDNVNNDVAHAVDVNNNNN

In [ ]:
# Bob infers the raw key.
# keyBob should be a string of n characters.
# Use the convention '0', '1', '-'=invalid measurement

# Modified to accomodate Psi-
keyBob = ""
for i in range(n):
    if basisBob[i] == '+':
        if outcomeBob[i] == 'H':
            keyBob += '1'
        elif outcomeBob[i] == 'V':
            keyBob += '0'
        else:
            keyBob += '-'
    elif basisBob[i] == 'x':
        if outcomeBob[i] == 'D':
            keyBob += '1'
        elif outcomeBob[i] == 'A':
            keyBob += '0'
        else:
            keyBob += '-'
print("keyBob      = " + keyBob)

keyBob      = --010010-1--1-1--0----000-11--1--0-0--1-0----101-00-0-00000----0-1--11010------00-----100---1-0-011101----10----1-1-001--10---01--1-011-1--0---1-00-10---1-01-0--110--1---0---1--001--0-10-----01-0-10-11-0--100---00--10-1--0-100-101--0-00-1--1-1-1-000--0-0-01--0--01110---11-100-1-1010-11-1-0-010-111-0--001-------1--1--0-------1--010-101-00-11-1--0--11101-0-----001-10-0-0--101100110---001-001-000--01--01----1-01011-10-011-0--1--1-0-00111-000-0-11-01---0--0-1-1101111-11-1000-01----0----0-111-1---0-1-0-10--0---0--1------1--01-1--0-----11101---1--------01--1---10----00-00-1---1----01110110--101-0-0-----0-00-001---01000-0-01-11-11100-0-010-1--1-0--00110-0------011--0-00--1----0-0--1--1---1111000--0-0-00---0-0-0-0100--01--0-0-0--10--0-0001-0-0-1011110-0-011-0-0--1--101-1--0--0-10-10------1-01----11-1011--0-1-1-0-00011--01-1-1-1-10-1-0110-11--010100---1-1110-0-10-0-1--00--1100--1-0-10--1----0-1--1-0-----10--01-100-0-0-1---01-101--11-1-1----001-1--1---0--1-0-0--0-1-0--10010010-----

In [ ]:
# -----------------------------------------------------------
# Alice and Bob now publicly announce which bases they chose.
# Bob also announces which of his measurements were invalid.
# -----------------------------------------------------------

In [ ]:
# Alice & Bob ----------------------------------------------------------

# Alice and Bob extract their sifted keys.
# siftedAlice and siftedBob should be strings of length n.
# Use the convention '0', '1', ' '=removed
siftedAlice = ""
siftedBob   = ""
# TODO: Put your code here.
for index in range(n):
    if keyAlice[index] == '-' or keyBob[index] == '-':
        siftedAlice += ' '
        siftedBob += ' '
    elif basisAlice[index] != basisBob[index]:
      siftedAlice += ' '
      siftedBob += ' '
    else:
        siftedAlice += keyAlice[index]
        siftedBob += keyBob[index]
print("siftedAlice = " + siftedAlice)
print("siftedBob   = " + siftedBob)

siftedAlice =   0  010    1    0     00  1  1  0 0  1 0    101 00 0 00000      1  1  10      00     100   1 0   1101    1     1 1  0    0   01  1 0   1  0   1 00 10   1 01 0    0  1   0   1  00   0 1      01 0 1  1  0   00   00  10 1  0  00 101    00 1      1 000  0 0  1  0  011     11 10  1  01     1 0  1  111                 1  0          010 10     11        11 1        0  10 0 0    110011    0 1 001 000   1  0       0101  10 011 0  1    0 00  1 000 0 11 0       0 1 1 0111  11 100        0    0 111         0 10  0   0  1      1  0     0     11101            01  1         0  00 1   1     11  11   101 0 0     0 00  01   0100  0 01 11  1100 0 010 1  1    0 110 0      011  0  0  1      0  1  1   1  100        0     0 0 0100  0   0   0   0  0 0001 0 0 10 1110   011 0 0  1  1   1  0  0 1  1         01    11 1  1      1 0 0 011  01 1     10 1  11   1  01010      1110 0 1       00  110     0 10  1    0 1  1       10     10  0 0 1   0  1    11   1    001 1      0    0 0  0 1 0  10010010     

In [ ]:
# Alice and Bob use a portion of their sifted keys to estimate the quantum bit error rate (QBER).
# sampleIndex should be a string of n characters.
# Use the convention '0'=ignored, '1'=sampled
# The QBER is the fraction of mismatches within the sampled portion.
# For large samples, it should be close to the actual QBER,
# which Alice and Bob, of course, do not know.
sampleIndex = ""
sampledBobQBER = 0
# TODO: Put your code here.
numError = 0
numTotal = 0

# out of non-removed sifted key, 50/50 chance to sample it
for i in range(n):
    if siftedAlice[i] == ' ' or random.randint(0, 1) == 0:
        sampleIndex += '0'
    else:
        sampleIndex += '1'
        numTotal += 1

#if sampling and it does not match add one to the number of errors
for i in range(n):
    if sampleIndex[i] == '1' and siftedAlice[i] != siftedBob[i]:
       numError += 1

sampledBobQBER = numError / numTotal
print(str(sampledBobQBER) + " QBER")
print("sampleIndex = "+sampleIndex)

0.0 QBER
sampleIndex = 00100001000000000000000110010000000000101000010101000001000000000100100010000001000000000000001000100100000000001010010000000000001010001000000000000000010100000001000000100000000000001000000100100000000000000000000010100000000110000001010000001000100100000000000110000001010001000000000001001000110000000000000000010000000000000011011000000100000000100000000000100110101000011101100001010100010000000000000000010100100000010000000000100101000001100000000010100011010011000100000000000000010100000000000000010001001000000000100000000000010000000000000001000000000000010001010000000000000110000110101000001001000100011110010010100010000000110100000001000101000000111001001000000000000000100000001000000000000000000011110000001000000010010001100000000001000001001010000010001000000000000000000000100000101001000000001000001001001000000001001100000001110000000100100010000000000000000000001000000001000010000000100000000001000000000010000100000000010001000000000000010000000001000

In [ ]:
# Alice and Bob remove the portion of their sifted keys that was sampled.
# Since a portion of the sifted key was publicly revealed, it cannot be used.
# secureAlice and secureBob should be strings of length n.
# Use the convention '0', '1', ' '=removed
secureAlice = ""
secureBob = ""
# TODO: Put your code here.
for i in range (n):
  # if we sample it is removed
  if sampleIndex[i] == '0':
    secureAlice += ' '
    secureBob += ' '
  #if we don't sample we just pass it on
  else:
    secureAlice += siftedAlice[i]
    secureBob += siftedBob[i]
print("secureAlice = " +secureAlice)
print("secureBob =   "  + secureBob)

secureAlice =   0    0               00  1          1 0    1 1 0     0         1  1   0      0              0   1  1          1 1  0            1 0   1                1 0       0      0             1      0  0                     0 1        10      0 1      1   0  0           11      1 1   1           0  1   11                 1              10 10      1        1           0  10 0 0    110 11    0 1 0   0                 1 1  1      0          0  1 0     11         0 1   01 1  11   0               1 1               0   0  1         0            1               0             0   0 1             11    01 0 0     0  0   1   0100  0  1 1   1       10 1       0   0 0      011  0  0               1       0                   0100      0       0  0   01          1     1  0 0     1   1                     1     1 1  1        0     1  0  1        1  11       101       1  0   1                     1        0    1       1          0          1    1         0   1             0         1   0 10     

In [ ]:
# Alice and Bob make a hard determination whether the channel is secure.
# If it looks like there's something fishy, better hit the kill switch!
channelSecure = True # default value, to be changed to False if Eve suspected
# TODO: Put your code here.
if (sampledBobQBER > .0001):
  channelSecure = False

In [ ]:
# Eve infers the raw key.
# keyEve should be a string of n characters.
# Use the convention '0', '1', '-'=invalid measurement, ' '=not measured

# Also adjusted for Psi-
keyEve = ""
for index in range(len(outcomeEve)):
  if outcomeEve[index] == "H":
    keyEve += "1"
  elif outcomeEve[index] == "V":
    keyEve += "0"
  elif outcomeEve[index] == "D":
    keyEve += "1"
  elif outcomeEve[index] == "A":
    keyEve += "0"
  elif outcomeEve[index] == "N" or outcomeEve[index] == "M":
    keyEve += "-"
  else:
    keyEve += " "

print("keyEve     = " + keyEve)

keyEve     =   101   000  00 01101 1  10    0  11 0  10 0 0 0            0  11 0 001 110111   0 00      1   0100     0  11      11101 01 011     100   0 101      1 0   1   10001   1 0     10  01   0111  0 001  1  0110 0 10 111    1 001 0 10 1 001   001 0100 0   1     1 01 001  01 1  01 110  01 1000   111 11    110110 01    01  0  0 010100 1  11  01110 0 00011 0  10110101011000 1 110 01      1 0  10   0     11 1  00  00    00 01 1 01     00     0001  1 1 0 1 0 011 0 0   0 00 0   1 111 10        0 0   000  1 0    11      1    0001010 0 01     0    10   0  0 1 1 10 1 10001 101 1 1  0 1 0   010001  10    0    000 11 1011 0 0   111 1  00 00001 0 1       0 1001100      011   101 1  1  0  11 10  0 1   00   10 10111 0001011   0    101  11  0101 0    1 110   10   111 1 00     0 10101 0  00 0 110110 10 0 100011    10  01 0    011   10 0  1010 0    1 01 00 1     11 1010 0 0  01 1 0     10  1 00 1  110       000001 11   11 101  1   1 000 10 010 1  00  1 1    10000 00   00      0   1 011  1   110 0

In [ ]:
# Eve extracts her sifted key.
# Knowing what Alice and Bob have publically revealed, Eve
# now selects which portion of her sifted key to keep.
# stolenEve should be strings of length n.
# Use the '0', '1', ' '=removed
stolenEve = ""
# TODO: Put your code here.
# i have keyalice and keybob
# i add
for i in range(n):
    if keyEve[i] == '-' or keyEve[i] == ' ' or basisAlice[i] != basisEve[i]:
        stolenEve += ' '
    else:
        stolenEve += keyAlice[i]
print("stolenEve = " + stolenEve)

stolenEve =   0              0010          1   0    0  1 1 1            1   0   1   001 0      11          1  1         0        0    0   0     0     1   0      0     0   0   0   0       01   0   1  0  1 1 0       0    0  000       10   01 0 11    11     1 1   0       1  110     0  10 0     0        0    0      1    1      0       1      0  00      1 1  1  0     10 1 1 1 0 11    01               1   1               1     1  10 0 1               10  0 0 1 0   1           11       00                  1 1         0       0    1 1 10     0     1    01      1 0 0 01 0 0     0     0  1   1     11     1    1      1 00 01 0 1 1   0 0 0   1 1 110 1         1  110 11      1 0    10    0                           1        10    1    0        1  0      0 00    0    00  0 1      1 01  0     1 1  01  1      01 10                  0       1  0 0  1      1   1 0      0 0  1 1    1          01       0  0 1       1 111   0   00   0      0 111 0  1   0      0 0    0111   1               0 100  0   00  1 

In [ ]:
# ANALYSIS -------------------------------------------------------------

# Below is a standard set of metrics to evaluate each protocol.
# You need not change any of what follows.

# Compare Alice and Bob's sifted keys.
numMatchBob = 0
actualBobQBER = 0
secureKeyRateBob = 0
secureKeyLengthBob = 0
for i in range(len(secureAlice)):
    if secureAlice[i] != ' ':
       secureKeyLengthBob += 1
       if secureAlice[i] == secureBob[i]:
           numMatchBob += 1

# Compute the actual quantum bit error rate for Bob.
if secureKeyLengthBob > 0:
    actualBobQBER = 1 - numMatchBob / secureKeyLengthBob
else:
    actualBobQBER = float('nan')
# Compute the secure key rate, assuming each trial takes 1 microsecond.
secureKeyRateBob = (1-actualBobQBER) * secureKeyLengthBob / n * 1e6;

# Compare Alice and Eve's sifted keys.
numMatchEve = 0
actualEveQBER = 0
stolenKeyRateEve = 0
stolenKeyLengthEve = 0
for i in range(len(stolenEve)):
    if stolenEve[i] != ' ':
       stolenKeyLengthEve += 1
       if secureAlice[i] == stolenEve[i]:
           numMatchEve += 1
# Compute the actual quantum bit error rate for Eve.
if stolenKeyLengthEve > 0:
    actualEveQBER = 1 - numMatchEve / stolenKeyLengthEve
else:
    actualEveQBER = float('nan')
# Compute the stolen key rate, assuming each trial takes 1 microsecond.
stolenKeyRateEve = (1-actualEveQBER) * stolenKeyLengthEve / n * 1e6;


# DISPLAY RESULTS ------------------------------------------------------

print("")
print("basisAlice  = " + basisAlice)
print("basisBob    = " + basisBob)
print("basisEve    = " + basisEve)
print("")
print("keyAlice    = " + keyAlice)
print("keyBob      = " + keyBob)
print("keyEve      = " + keyEve)
print("")
print("siftedAlice = " + siftedAlice)
print("siftedBob   = " + siftedBob)
print("")
print("secureAlice = " + secureAlice)
print("secureBob   = " + secureBob)
print("stolenEve   = " + stolenEve)
print("")
if not channelSecure:
    secureKeyRateBob = 0;
    stolenKeyRateEve = 0;
    print("*********************************************")
    print("* ALERT! The quantum channel is not secure. *")
    print("*********************************************")
    print("")
print("sampledBobQBER = " + str(sampledBobQBER))
print("actualBobQBER  = " + str(actualBobQBER))
print("actualEveQBER  = " + str(actualEveQBER))
print("")
print("secureKeyRateBob = " + str(secureKeyRateBob/1000) + " kbps")
print("stolenKeyRateEve = " + str(stolenKeyRateEve/1000) + " kbps")

# Your goal is to maximize secureKeyRateBob and minimize stolenKeyRateEve.


basisAlice  = xxx++xxx+++x++x+x++xx+xxxx+x+xx+++xxx+x+x+++xxxxx+x+xxx+xx+x+xx+xx+++x++xx+xx+x+xx++xxxx+++++xxxx++xxx++xxxxxx+x+x+x+++xxxx+xx++x+++++xxx+xx+xx++xx+x+++xxx++++x++x++xx+x++x+x++xx+++++xxxxx+xxx+x+xx+xx+++++x+x+xx+x+x+xxx+xx+xxx++++xx++x+x++xxxx++x+++++xx+++++xxxx+xxxx+xxx+++++x+x+xxxx+xxx++xx++x+x+x+x+x++x++xxxxx+xxx++x++xx+++xxx+x+x++x+xx+x++xxx++xxx++x+x++xxxx++++xxx+x++++xx+xx+xx++x++x++++x+x+++x++xx++x+xx+x+x++++xxx+xx++xx+++xxx+++xx+xx+xxx+++xx+x+x+xxxx+xx+++++x++xxxx+++++++++++xx++xx+xx++++xxxxxx+++++xxx+++++xxxx+x+x+++xx+++x+xxx+xxx++++x+xxxx+++++x+x+x+++xx+++xxxxx++++++xx++++++x+x+x++xxxxxx++xx++xxxx++xx+xxx++x+x++x+x++xxxxx+++x++x+xx++x+++xx+xx+xx++xx++xxx+x+++++xxx+++x+xxxxx+xxx+xxxxx++xxxxxx+xx++xxxx+x++x+x+++x++xxx+xxx++x+xx++x+xxx+xx++xx+x+xx+xx+xxx+++xxxx++xxx+++x++xx++xxxxxx++x+++++++xxxx+++xx+x+++++x++xx++xxxx++++x+++++x+x++x+xx+xx+++xxx++++x++x++xxx++x+xxx+xx++x+xxxxx++x++x++xxx+++xxxx+x+xx+xx+++xxxxx+x+x+++++++xx++xxx+xxx++x++xxxx+++x+xxx++xxx+x+x+x+x+xx